In [4]:
import json
import config
from RiotRequest import RiotRequest
from RiotAPI import get_account_id, get_champion_json

api_key = config.api_key


############## 계정 key 관리하기 ##############

# summoner id 로 가져오고 account id로 저장하기 위함
def get_accountId(encryptedSummonerId):
    '''Summoner Id에서 account Id 가져오기'''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/summoner/v4/summoners/{0}?api_key={1}".format(encryptedSummonerId, api_key))
    try:
        return json_data['accountId']
    except TypeError as e:
        print(e)
        print(encryptedSummonerId)
        return encryptedSummonerId

############## 각 division 별 summoner Id의 리스트를 반환하는 함수 ##############
    
def get_grandmaster_smId(queue):
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
    smId = [Id['summonerId'] for Id in json_data['entries']]
    return smId

def get_challenger_smId(queue):
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
    smId = [Id['summonerId'] for Id in json_data['entries']]
    return smId

def get_master_smId(queue):
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
    smId = [Id['summonerId'] for Id in json_data['entries']]
    return smId


############## 숙련도 정보 추출하기 ##############

def get_champion_masteries(encryptedSummonerId):
    '''
    해당 Summoner의 숙련도 전체 정보
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{0}?api_key={1}".format(encryptedSummonerId, api_key))
    
    try:
        return json_data
    except:
        print(encryptedSummonerId)
        return encryptedSummonerId


def get_champion_points(encryptedSummonerId):

    champion_masteries = get_champion_masteries(encryptedSummonerId)
    champion_points_top5 = {}
    errorId = []
    
    try:
        for j in range(5):
            champion_points_top5[champion_masteries[j]['championId']] = champion_masteries[j]['championPoints']
        
        return champion_points_top5
    
    except IndexError as e:
        print(e)
        print(encryptedSummonerId)
        for j in range(len(champion_masteries)):
            champion_points_top5[champion_masteries[j]['championId']] = champion_masteries[j]['championPoints']
        return champion_points_top5
    
    except TypeError as e:
        print(e)
        print(encryptedSummonerId)
        return champion_points_top5

In [ ]:
def get_most_champ_challenger():
    CGsmId = get_challenger_smId("RANKED_SOLO_5x5")

    champion_points_CG = {} # {summmoner id : ordered dict of points of masteries}

    for smId in CGsmId:
        accId = get_accountId(smId)
        champion_points_CG[accId] = get_champion_points(smId)
    
    return champion_points_CG